# Patient Satisfaction

## Import Libraries and Datasets

In [1]:
# Import libraries
import pandas as pd

In [18]:
# Read the data from different years into DataFrames
df = pd.read_csv("../data/raw/HCAHPS-Hospital.csv")
df['Year']=2023

# Sample data of dataframe:
df.head()

C:\Users\vivas\AppData\Local\Temp\ipykernel_25452\337868718.py:2: DtypeWarning: Columns (12,14,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/HCAHPS-Hospital.csv")


,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,HCAHPS Measure ID,HCAHPS Question,...,HCAHPS Answer Percent,HCAHPS Answer Percent Footnote,HCAHPS Linear Mean Value,Number of Completed Surveys,Number of Completed Surveys Footnote,Survey Response Rate Percent,Survey Response Rate Percent Footnote,Start Date,End Date,Year
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_A_P,"Patients who reported that their nurses ""Alway...",...,74,NaN,Not Applicable,544,NaN,15,NaN,04/01/2022,03/31/2023,2023
1,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_SN_P,"Patients who reported that their nurses ""Somet...",...,8,NaN,Not Applicable,544,NaN,15,NaN,04/01/2022,03/31/2023,2023
2,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_U_P,"Patients who reported that their nurses ""Usual...",...,18,NaN,Not Applicable,544,NaN,15,NaN,04/01/2022,03/31/2023,2023
3,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_LINEAR_SCORE,Nurse communication - linear mean score,...,Not Applicable,NaN,89,544,NaN,15,NaN,04/01/2022,03/31/2023,2023
4,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,H_COMP_1_STAR_RATING,Nurse communication - star rating,...,Not Applicable,NaN,Not Applicable,544,NaN,15,NaN,04/01/2022,03/31/2023,2023


## Data Preparation

### Rename Measure IDs

In [4]:
print('Cleaned dataset for 2023, number of rows and columns: ')
print(df.shape)
print('')
# Define the replacement mappings
rename_mappings = {
    "H_COMP_1_": "NURSE_COMM_",
    "H_COMP_2_": "DOCTOR_COMM_",
    "H_COMP_3_": "STAFF_RESPON_",
    "H_COMP_5_": "MEDICINE_",
    "H_COMP_6_": "DISCHARGE_INFO_",
    "H_COMP_7_": "CARE_TRANSIT_",
    "H_CLEAN_HSP_":"H_CLEAN_",
    "H_QUIET_HSP_":"H_QUIET_"
}
df["HCAHPS Measure ID"] = df["HCAHPS Measure ID"].replace(rename_mappings, regex=True)

Cleaned dataset for 2023, number of rows and columns: 
(447516, 23)



### Add "HCAHPS Answer" Column

In [5]:
# Create 'Value' column
def get_value(row):
    
    if row['HCAHPS Measure ID'].endswith('_LINEAR_SCORE'):
        return row['HCAHPS Linear Mean Value']
    elif row['HCAHPS Measure ID'].endswith('_STAR_RATING'):
        return row['Patient Survey Star Rating']
    else:
        return row['HCAHPS Answer Percent']

# Apply the function to create the new column
df['HCAHPS Answer'] = df.apply(get_value, axis=1)

### Generate Pivot Table

In [6]:
# Get unique values on Measure ID
pivot_cols = list(df['HCAHPS Measure ID'].unique())

# Get the group by columns
group_by = group_by= list(df.columns[:8])+['Number of Completed Surveys', 'Survey Response Rate Percent','Year']
pivot_cols = group_by + pivot_cols

# Pivot the data
pivot_df = df.pivot_table(index=group_by,
                 columns='HCAHPS Measure ID', 
                 values='HCAHPS Answer',
                 aggfunc=lambda x: x.mode().iloc[0])

# Reset index to make Facility ID a column again
pivot_df.reset_index(inplace=True)
# Re-order the columns
pivot_df=pivot_df[pivot_cols]

# Display the pivoted DataFrame
pivot_df.head()

HCAHPS Measure ID,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Number of Completed Surveys,Survey Response Rate Percent,...,H_HSP_RATING_7_8,H_HSP_RATING_9_10,H_HSP_RATING_LINEAR_SCORE,H_HSP_RATING_STAR_RATING,H_RECMND_DN,H_RECMND_DY,H_RECMND_PY,H_RECMND_LINEAR_SCORE,H_RECMND_STAR_RATING,H_STAR_RATING
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,544,15,...,20,72,88,3,4,75,21,90,4,3
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,824,18,...,25,65,85,3,7,62,31,84,3,3
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 629-1000,1503,19,...,24,59,82,2,13,56,31,79,2,2
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,189,24,...,24,69,88,3,5,67,28,86,4,3
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,46,26,...,11,84,Not Available,Not Available,8,75,17,Not Available,Not Available,Not Available


### Convert to Numeric Values

In [7]:
# Replace any custom representation of missing values with NA
pivot_df.replace("Not Available", pd.NA, inplace=True)

# Convert numeric columns to integer type
pivot_df = pivot_df.apply(pd.to_numeric, errors='ignore')
pivot_df['ZIP Code'] = pivot_df['ZIP Code'].astype(object)
numeric_col_names = pivot_df.select_dtypes(include='number').columns
pivot_df[numeric_col_names] = pivot_df[numeric_col_names].astype('Int64')
del numeric_col_names

print(pivot_df.info(verbose=True))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4812 entries, 0 to 4811
Data columns (total 104 columns):
 #    Column                        Dtype 
---   ------                        ----- 
 0    Facility ID                   object
 1    Facility Name                 object
 2    Address                       object
 3    City/Town                     object
 4    State                         object
 5    ZIP Code                      object
 6    County/Parish                 object
 7    Telephone Number              object
 8    Number of Completed Surveys   Int64 
 9    Survey Response Rate Percent  Int64 
 10   Year                          Int64 
 11   NURSE_COMM_A_P                Int64 
 12   NURSE_COMM_SN_P               Int64 
 13   NURSE_COMM_U_P                Int64 
 14   NURSE_COMM_LINEAR_SCORE       Int64 
 15   NURSE_COMM_STAR_RATING        Int64 
 16   H_NURSE_RESPECT_A_P           Int64 
 17   H_NURSE_RESPECT_SN_P          Int64 
 18   H_NURSE_RESPECT_U_P       

### Handle missing data

In [8]:
## Show Initial Information
print("Number of hospitals in dataset: ")
print(len(pivot_df),"\n")

print("Null values per columns: ")
#print(pivot_df.isnull().sum())
print(pivot_df.isnull().sum()[pivot_df.isnull().sum() > 0],"\n")

## Exclude Facilities that do not have any answers
cleaned_pivot_df = pivot_df[~pivot_df['Number of Completed Surveys'].isnull()]

print("Null values per columns after excluding all nulls: ")
#swith pd.option_context("display.max_rows", None):
print(cleaned_pivot_df.isnull().sum()[cleaned_pivot_df.isnull().sum() > 0],"\n")

print("Number of hospitals after excluding all nulls: ")
print(len(cleaned_pivot_df),"\n")

## Exclude Facilities that have less than 50 number of completed surveys (too many columns with empty value)
cleaned_pivot_df = cleaned_pivot_df[cleaned_pivot_df['Number of Completed Surveys']>50]
cleaned_pivot_df = cleaned_pivot_df[cleaned_pivot_df['H_NURSE_RESPECT_A_P'].notnull()]

print("Null values per columns after excluding all nulls: ")
#with pd.option_context("display.max_rows", None):
print(cleaned_pivot_df.isnull().sum()[cleaned_pivot_df.isnull().sum() > 0],"\n")

print("Number of hospitals after excluding all nulls: ")
print(len(cleaned_pivot_df),"\n")

Number of hospitals in dataset: 
4812 

Null values per columns: 
HCAHPS Measure ID
Number of Completed Surveys      716
Survey Response Rate Percent     716
NURSE_COMM_A_P                   716
NURSE_COMM_SN_P                  716
NURSE_COMM_U_P                   716
                                ... 
H_RECMND_DY                      716
H_RECMND_PY                      716
H_RECMND_LINEAR_SCORE           1554
H_RECMND_STAR_RATING            1554
H_STAR_RATING                   1554
Length: 95, dtype: int64 

Null values per columns after excluding all nulls: 
HCAHPS Measure ID
NURSE_COMM_LINEAR_SCORE      838
NURSE_COMM_STAR_RATING       838
H_NURSE_RESPECT_A_P          331
H_NURSE_RESPECT_SN_P         331
H_NURSE_RESPECT_U_P          331
                            ... 
H_HSP_RATING_LINEAR_SCORE    838
H_HSP_RATING_STAR_RATING     838
H_RECMND_LINEAR_SCORE        838
H_RECMND_STAR_RATING         838
H_STAR_RATING                838
Length: 64, dtype: int64 

Number of hospitals af

In [9]:
# Create an empty DataFrame to store the results
linear_score_comparison = pd.DataFrame()

# Iterate through each feature
for feature in [col.split('_LINEAR_SCORE')[0] for col in cleaned_pivot_df.columns if col.endswith('_LINEAR_SCORE')]:
    # Depending on the feature
    if feature == 'DISCHARGE_INFO':
        calculated_linear_score = cleaned_pivot_df['DISCHARGE_INFO_Y_P'].round();
    elif feature =='H_HSP_RATING':
        # Calculate the linear score using the specified formula
        calculated_linear_score = (cleaned_pivot_df[f'{feature}_0_6'] *0.45 +
                                cleaned_pivot_df[f'{feature}_7_8'] * 0.8 +
                                cleaned_pivot_df[f'{feature}_9_10'] * 0.95).round()
    else:
        if f'{feature}_A_P' in cleaned_pivot_df.columns:
            A_P_column = f'{feature}_A_P'
            U_P_column = f'{feature}_U_P'
            SN_P_column = f'{feature}_SN_P'
        # Check if _SA columns exists
        elif f'{feature}_SA' in cleaned_pivot_df.columns:
            A_P_column = f'{feature}_SA'
            U_P_column = f'{feature}_A'
            SN_P_column = f'{feature}_D_SD'
        # Check if _DY columns exists
        elif f'{feature}_DY' in cleaned_pivot_df.columns:
            A_P_column = f'{feature}_DY'
            U_P_column = f'{feature}_PY'
            SN_P_column = f'{feature}_DN'
        else:
            print(f"{feature}_AP does not exist for feature {feature}. Skipping...")
            continue

        # Calculate the linear score using the specified formula
        calculated_linear_score = (cleaned_pivot_df[A_P_column] +
                                cleaned_pivot_df[U_P_column] * 2/3 +
                                cleaned_pivot_df[SN_P_column] * 1/6+0.6).round()
    # Fill NA values in the original DataFrame with calculated linear scores
    cleaned_pivot_df[f'{feature}_LINEAR_SCORE'].fillna(calculated_linear_score, inplace=True)
    
    # Create a DataFrame to store the comparison results for the current feature
    feature_comparison = pd.DataFrame({
        'Actual_Score': cleaned_pivot_df[f'{feature}_LINEAR_SCORE'],
        'Calculated_Score': calculated_linear_score,
        #f'{feature}_Difference': calculated_linear_score - cleaned_pivot_df[f'{feature}_LINEAR_SCORE']
        'Difference': calculated_linear_score - cleaned_pivot_df[f'{feature}_LINEAR_SCORE']
    })
    
    # Append the comparison results to the overall DataFrame
    linear_score_comparison = pd.concat([linear_score_comparison, feature_comparison], axis=0,ignore_index=True)
    #linear_score_comparison = pd.concat([linear_score_comparison, feature_comparison], axis=1)

#Drop with NA values
#linear_score_comparison=linear_score_comparison.dropna(subset=['Actual_Score'])
# Display the comparison results
print(linear_score_comparison.mean())

print("Null values per columns after Filling null values: ")
#with pd.option_context("display.max_rows", None):
print(cleaned_pivot_df.isnull().sum()[cleaned_pivot_df.isnull().sum() > 0],"\n")

print("Number of hospitals after Filling null values: ")
print(len(cleaned_pivot_df),"\n")


Actual_Score         84.96321
Calculated_Score    84.953479
Difference          -0.009731
dtype: Float64
Null values per columns after Filling null values: 
HCAHPS Measure ID
NURSE_COMM_STAR_RATING        493
DOCTOR_COMM_STAR_RATING       493
STAFF_RESPON_STAR_RATING      493
MEDICINE_STAR_RATING          493
DISCHARGE_INFO_STAR_RATING    493
CARE_TRANSIT_STAR_RATING      493
H_CLEAN_STAR_RATING           493
H_QUIET_STAR_RATING           493
H_HSP_RATING_STAR_RATING      493
H_RECMND_STAR_RATING          493
H_STAR_RATING                 493
dtype: int64 

Number of hospitals after Filling null values: 
3751 



### Reduced pivot table

In [10]:
# Filter columns that end with "LINEAR_SCORE"
linear_score_columns = [col for col in cleaned_pivot_df.columns[11:] if col.endswith('LINEAR_SCORE')]

# Create a new DataFrame with selected columns
reduced_df = pd.concat([cleaned_pivot_df.iloc[:, :11].drop(columns=cleaned_pivot_df.columns[[2,5,6,7]]), cleaned_pivot_df[linear_score_columns].rename(columns=lambda x: x.replace('_LINEAR_SCORE', ''))], axis=1)
reduced_df

print("Null values per columns after droping non-necessary columns: ")
print(reduced_df.isnull().sum()[reduced_df.isnull().sum() > 0],"\n")

print(reduced_df.info(),"\n")


Null values per columns after droping non-necessary columns: 
Series([], dtype: int64) 

<class 'pandas.core.frame.DataFrame'>
Index: 3751 entries, 0 to 4808
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Facility ID                   3751 non-null   object
 1   Facility Name                 3751 non-null   object
 2   City/Town                     3751 non-null   object
 3   State                         3751 non-null   object
 4   Number of Completed Surveys   3751 non-null   Int64 
 5   Survey Response Rate Percent  3751 non-null   Int64 
 6   Year                          3751 non-null   Int64 
 7   NURSE_COMM                    3751 non-null   Int64 
 8   DOCTOR_COMM                   3751 non-null   Int64 
 9   STAFF_RESPON                  3751 non-null   Int64 
 10  MEDICINE                      3751 non-null   Int64 
 11  DISCHARGE_INFO                3751 non-null   Int6

### Add Region Information

In [16]:
# Import states by region
state_region_df = pd.read_csv("../data/raw/state regions.csv")

# Merge state_stats with state_region_df to include region information
completed_df = pd.merge(reduced_df, state_region_df, how='left', on='State')
# Rename State Name column
completed_df = completed_df.rename(columns={"Name": "State Name"})

### Rename final Categories

In [12]:
# Create a dictionary to map original column names to desired names
rename_mappings = {
    'NURSE_COMM':'Nurse Communication',
    'DOCTOR_COMM':'Doctor Communication',
    'STAFF_RESPON':'Staff Responsiveness',
    'MEDICINE':'Medicine Communication',
    'DISCHARGE_INFO':'Discharge Information',
    'CARE_TRANSIT':'Care Transition',
    'H_CLEAN':'Cleanliness',
    'H_QUIET':'Quietness',
    'H_HSP_RATING':'Rating Score',
    'H_RECMND': 'Recommendation'
}

# Rename the columns using the dictionary
completed_df = completed_df.rename(columns=rename_mappings, index=rename_mappings)

completed_df.head()

,Facility ID,Facility Name,City/Town,State,Number of Completed Surveys,Survey Response Rate Percent,Year,Nurse Communication,Doctor Communication,Staff Responsiveness,Medicine Communication,Discharge Information,Care Transition,Cleanliness,Quietness,Rating Score,Recommendation,State Name,Region,Division
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,DOTHAN,AL,544,15,2023,89,90,81,78,86,82,85,85,88,90,Alabama,South,East South Central
1,010005,MARSHALL MEDICAL CENTERS,BOAZ,AL,824,18,2023,90,92,74,75,86,79,80,85,85,84,Alabama,South,East South Central
2,010006,NORTH ALABAMA MEDICAL CENTER,FLORENCE,AL,1503,19,2023,88,88,75,70,84,77,73,84,82,79,Alabama,South,East South Central
3,010007,MIZELL MEMORIAL HOSPITAL,OPP,AL,189,24,2023,90,94,86,76,85,80,79,86,88,86,Alabama,South,East South Central
4,010011,ST. VINCENT'S EAST,BIRMINGHAM,AL,2023,33,2023,90,90,85,76,86,81,82,83,88,87,Alabama,South,East South Central


## Export Processed Data

In [14]:
# Generate CSV file
completed_df.to_csv('../data/processed/completed_data.csv', index=False)